In [ ]:
/// 一键打印 治疗单 处方单 输液单 注射单
function PrintClick() {
	PrintDo('Y');
}
function PrintDo(printFlag){
	if (printFlag=="EMR"){
	}else{
		var rtn=prepareHandle(function(options,isArray){
			if (options["node"]["value"]=="MZBL"){
				if ($("#PrintDataPreView").length==0){
					printFlag="N";
					onlyPreviewMZBL();
				}else{
					
				}
			}else{
				if(isArray){
					options["ReadyJson"]=options["ReadyJson"]["data"];
				}
				var pd=null;
				if(printDeviceCom && (pd=printDeviceCom.getSelected())){
					options["printDevice"]=pd["name"];
				}
				if($paperSize  && $paperSize.val()){
					options["paperSize"]=$paperSize.val()
				}
				//options["printCopies"]=$printCopies!=null?$printCopies.val()||1:1;
				options["printCopies"]=1;
				opdoc.print.common.print(options);
				
			}
			
		},"Print",printFlag);
	}
	if (printFlag=="Y"){
		if (rtn!=false){
			needPrintConfig={}; //打印之后清空需要打印配置
			destroyDialog('PrintDataPreView');
			loadOrdTableData();
			if (parent.window.CompleteRecAdm) {
				parent.window.CompleteRecAdm();
			}else if(parent.window.PrintAllAfterClick){
				parent.window.PrintAllAfterClick();
			}
		}
	}
}
var needPrintConfig={};
function prepareHandle(callBack,type,printFlag){
	needPrintConfig=getNeedPrint();
	if ($.isEmptyObject(needPrintConfig["selectedOE"])){
		if (!needPrintConfig["menuNodes"]["MZBL"]){
			$.messager.alert("提示","请选择医嘱!");
			if (type=="PreView") {
				$("#PrintDataPreView").dialog('close');
			}
			return false;
		}
	}
	if (type=="Print"){
		if ((needPrintConfig["menuNodes"]["CFD"])||(needPrintConfig["menuNodes"]["CFZ"])){
			/*
			if (!checkLocType()) {
				//needPrintConfig={}; 
				return false;
			}
			*/
		}
	}
	
	for(var mv in needPrintConfig["menuNodes"]){
		var menuNode=needPrintConfig["menuNodes"][mv];
		var noeValue=menuNode['value'];
		var printmethod=menuNode['printmethod'];
		if (noeValue!="MZBL"){
				if ((type=="PreView")||(printFlag=="N")) { //printFlag 为N代表 是直接点击打印按钮弹出病历界面
					var printmethod=resolveConfigMethod(menuNode["printmethod"],menuNode,needPrintConfig["menuOEMap"][mv],{},type);
				}else{
					var printmethod=resolveConfigMethod(menuNode["printmethod"],menuNode,needPrintConfig["menuOEMap"][mv],needPrintConfig["selectedListRowId"],type);
				}
				if(printmethod==null) continue;
				var argN=0,a=0;
				for(var a in printmethod["args"]){
					argN=argN+1;
				}
				//配置xml对应ID
				var tempId=$.cm({
					ClassName:"web.DHCDocPrescript",
					MethodName:"GetXMLTemplateId",
					dataType:"text",
					XMLTemplateName:menuNode["previewxmlname"]
				},false);
				var options={
					node:menuNode,
					"templateId":tempId
				};
				$.ajax("opdoc.request.method.csp",{
					type : "POST",
					dataType:"JSON",
					async: false, //此处必须设置为同步，否则预览标题与正文不符
					data:opdoc.util.genQueryArg({
						ClassName : printmethod["class"],//"DHCDoc.OPDoc.TreatPrint",
						MethodName : printmethod["method"]//"PrintOrder"
					}, argN, printmethod["args"]),
					success:function(respData) {
						respData=respData["result"];
						if($.type(respData) === "array"){
							for(var i=0,len=respData.length;i<len;++i){
								if(respData[i]["listRowId"]==undefined){
									console.log("when return array each item must be set listRowId value!");
									return;
								}
								var Onepagenum=1;
								options["ReadyJson"]=respData[i];
							    //若后台返回的xml模板名不为空,则根据返回模板展示
								if ((respData[i]["templateId"]!="")&&(respData[i]["templateId"]!=undefined)){
									options["templateId"]=respData[i]["templateId"];
								}else if ((respData[i]["PrintTemp"]!="")&&(respData[i]["PrintTemp"]!=undefined)){
									var JsontempId=$.cm({
										ClassName:"web.DHCDocPrescript",
										MethodName:"GetXMLTemplateId",
										dataType:"text",
										XMLTemplateName:respData[i]["PrintTemp"]
									},false);
									options["templateId"]=JsontempId;
								}else{
									options["templateId"]=tempId;
								}
								if (options['node']['value'].indexOf("CF")>=0){
									if (respData[i]["PrintTemp"].indexOf("Ver")<0){
										var Onepagenum=12; //处方打印一页打印行数
									}else{
										var Onepagenum=16;
									}
								}
								
								options["printPageNums"]=Onepagenum;
								callBack(options,true); //renderPrivewPanel
							}
						}else{
							options["ReadyJson"]=respData;
							callBack(options,false);
						}
						
					},
					error:function(XMLHttpRequest, textStatus, errorThrown) {
						console.log(XMLHttpRequest);
					}
				});
		}else{
			var urlObject=menuNode['urlObject'];
			var urlStyle=menuNode['urlStyle'];
			if (urlStyle=="iframe"){
				var InstancePrintedStatus=GetInstancePrintedStatus();
				if (JSON.parse(InstancePrintedStatus).hasNoPrinted==0){
					if (Object.keys(needPrintConfig["menuNodes"]).length==1) {
						$.messager.alert("提示","没有需要打印的病历,请核实病历是否签名或已打印!");
						if (type=="PreView") {
							$("#PrintDataPreView").dialog('close');
						}
						return false;
					}
					continue;
				}
				var newurl=replaceLinkParams(urlObject);
				var options={
					node:menuNode,
					newurl:newurl,
					ReadyJson:{"listRowId":ServerObj.EpisodeID+"_"+menuNode['value'],"checkTitle":''}
				};
				callBack(options,true); 
			}
		}
	}	
}
function resolveConfigMethod(configMethod,node,oeList,selectedListRows,type){
	if(!configMethod) return null;
	var i=configMethod.indexOf("?");
	var fullName=configMethod.substring(0,i==-1?configMethod.length:i);
	var paramMethod=i==-1?null:configMethod.substring(i+1);
	i=fullName.lastIndexOf(".");
	if(i==-1) return null;
	return {
		"class":fullName.substring(0,i),
		"method":fullName.substring(i+1),
		"args":paramMethod!=null?eval((paramMethod+"(node,oeList,selectedListRows,type)")):{}
	}
}
function prescriptPrintDataArgs(node,selectedOE,selectedListRows,type){
	var AdmDate = tkMakeServerCall('web.DHCDocMain','GetAdmDate',GlobalObj.EpisodeID);
	return {
		"Arg1":GlobalObj.EpisodeID,
		"Arg2":opdoc.util.joinJsonPro(selectedOE),
		"Arg3":opdoc.util.joinJsonPro(selectedListRows,"^"),
		"Arg4":node["value"],
		"Arg5":type,
		"Arg6":AdmDate,
		"Arg7":''
	};
}
var menuItemValueMap = {};
var printDeviceCom=null;
var $paperSize=null;
var allOEArr=new Array;
var waitCancelOEArr=new Array;
var m_IDCredTypePlate="01"; //身份证代码字段
var OrderPrescNo="";

function Node(init,node) {
    var cfz = {};
    init.forEach(function(item) {
        if (item.config.menuOEConfig[node]) {
            var obj = item.config.menuOEConfig[node]
            for (var key in obj) {
               cfz[key] = true;
            }
        }
    });
    return cfz;
}

function InPatPrescPrint() {
	window.open ("ipdoc.prescprintall.csp", "newwindow", "height=750, width=1450, toolbar =no,top=100,left=300,menubar=no, scrollbars=yes, resizable=yes, location=no,status=no");
}
function getNeedPrint() {
	var menuItems = {
  "CFZ": {
    "id": "3",
    "text": "处方【正】",
    "state": "open",
    "pId": "1",
    "checked": false,
    "ischecked": true,
    "nocheck": true,
    "expandMethodForLoadingTable": "",
    "urlObject": "",
    "clickevent": "",
    "value": "CFZ",
    "urlStyle": "",
    "previewxmlname": "DHCOutPrescXYPrt",
    "printmethod": "DHCDoc.OPDoc.TreatPrint.PrescriptPrintData?prescriptPrintDataArgs",
    "domId": "_hisui_tree_2",
    "target": {}
  },
  "SYDO": {
    "id": "5",
    "text": "输液单",
    "state": "open",
    "pId": "1",
    "checked": false,
    "ischecked": true,
    "nocheck": true,
    "expandMethodForLoadingTable": "",
    "urlObject": "",
    "clickevent": "",
    "value": "SYDO",
    "urlStyle": "",
    "previewxmlname": "DHCDocTransFusion",
    "printmethod": "DHCDoc.OPDoc.TreatPrint.PrescriptPrintData?prescriptPrintDataArgs",
    "domId": "_hisui_tree_3",
    "target": {}
  },
  "ZSDO": {
    "id": "6",
    "text": "注射单",
    "state": "open",
    "pId": "1",
    "checked": false,
    "ischecked": false,
    "nocheck": true,
    "expandMethodForLoadingTable": "",
    "urlObject": "",
    "clickevent": "",
    "value": "ZSDO",
    "urlStyle": "",
    "previewxmlname": "DHCDocTransFusion",
    "printmethod": "DHCDoc.OPDoc.TreatPrint.PrescriptPrintData?prescriptPrintDataArgs",
    "domId": "_hisui_tree_4",
    "target": {}
  },
  "ZLDO": {
    "id": "9",
    "text": "治疗处置单",
    "state": "open",
    "pId": "1",
    "checked": false,
    "ischecked": true,
    "nocheck": true,
    "expandMethodForLoadingTable": "",
    "urlObject": "",
    "clickevent": "",
    "value": "ZLDO",
    "urlStyle": "",
    "previewxmlname": "DHCDocExec",
    "printmethod": "DHCDoc.OPDoc.TreatPrint.PrescriptPrintData?prescriptPrintDataArgs",
    "domId": "_hisui_tree_6",
    "target": {}
  }
};
	var init = {}
	var AdmDate = tkMakeServerCall('web.DHCDocMain','GetAdmDate',GlobalObj.EpisodeID);
	
	var init = $.cm({
		ClassName:"DHCDoc.OPDoc.TreatPrint",
		MethodName:"QueryOEForPrintJson",
		episodeID:GlobalObj.EpisodeID,
		menuItems:'CFZ,SYDO,ZSDO,ZLDO',
		StDate:AdmDate,
		EndDate:'',
		OrdComStatus:'UnPay'
	},false);
	
	var end = {}
	end.CFZ = Node(init,'CFZ');
	end.SYDO = Node(init,'SYDO');
	end.ZSDO = Node(init,'ZSDO');
	end.ZLDO = Node(init,'ZLDO');
	
	var selectedOE = {};
	for (var key in end) {
		var obj = end[key]
		for (var ele in obj) {
	    	selectedOE[ele] = true;
	    }
    }
	return {
		"menuNodes" : menuItems,
		"selectedOE" : selectedOE,
		"menuOEMap" : end,
		"selectedListRowId" : {} //!$.isEmptyObject(needPrintConfig["selectedListRowId"])?needPrintConfig["selectedListRowId"]:{}
	};
}